In [77]:
import networkx as nx
import math
import time
import progressbar
import tqdm
import random
import collections
import matplotlib.pyplot as plt
%matplotlib inline
from multiprocessing import Pool, cpu_count, Queue, Manager
%reload_ext Cython
import numpy as np

SyntaxError: invalid syntax (<ipython-input-77-801fa9eed988>, line 11)

In [8]:
R = 10
N = 106770
T = 1
d = 2
threshold = R*T

In [79]:
def generate_label(i):
    label = []
    j = 0
    while j < T:
        l = random.randint(1, 2**(R+1)-1)
        if random.random() < 1.0/(1.2**(l.bit_length()-1)):
            labels[i][j][0] = int(l.bit_length()-1)
            labels[i][j][1] = int(l)
            j += 1
    return label
#labels = []
labels = np.ndarray(shape=(N,T,2), dtype=np.uint32)
for i in range(N):
    #labels.append(generate_label())
    generate_label(i)

In [80]:
G = nx.Graph()
G.add_node(range(N))
manager = Manager()
bar = progressbar.ProgressBar()
A = [[] for _ in labels]

In [84]:
%%cython 
#--annotate
cimport numpy as np
from __main__ import T, threshold, tqdm, Manager, random
from __main__ import labels as labels

cdef extern from "math.h":
    double exp(double x)
cdef extern int __builtin_clz (unsigned int x)

cdef double label_distance(int i1, int i2, unsigned int *L, int T, int c_threshold):
    cdef unsigned int ret = 0
    cdef unsigned int d1, l1, d2, l2, j
    cdef unsigned int indx1, indx2, height_delta
    for j in range(T):
        indx1 = j*2 + i1*T*2
        indx2 = j*2 + i2*T*2
        if L[indx2] > L[indx2]:
            d1 = L[indx1]
            l1 = L[indx1+1]
            d2 = L[indx2]
            l2 = L[indx2+1]
        else:
            d1 = L[indx2]
            l1 = L[indx2+1]
            d2 = L[indx1]
            l2 = L[indx1+1]
        height_delta = d2-d1
        l2 = l2 >> height_delta
        #tmp = l2 ^ l1
        #print(bin(tmp), tmp.bit_length()-1, __builtin_clz(tmp))
        #if(31 ^ __builtin_clz(((l2^l1) << 1) + 1) != tmp.bit_length()):
        #    print(tmp.bit_length() + __builtin_clz(((l2^l1) << 1) + 1))
        ret = ret + 2*(31 ^ __builtin_clz(((l2^l1) << 1) + 1)) + height_delta
    cdef double beta = 3.0
    #print(1.0/(exp(beta*(ret-c_threshold)) + 1))
    return 1/(exp(beta*(ret-c_threshold)) + 1)

Q = Manager().Queue()
def compute_neighbors(int i):
    cdef unsigned int c_T = T
    cdef unsigned int c_threshold = threshold
    cdef np.ndarray[unsigned int, ndim=3, mode="c"] L = labels
    cdef unsigned int *Lptr = &L[0,0,0]
    cdef int num_labels = len(labels)
    cdef int j
    for j in range(i+1,num_labels):
        dist = label_distance(i, j, Lptr, c_T, c_threshold)
        if random.random() < dist:
        #if False:
            Q.put((i,j))

In [85]:
compute_neighbors(0)

In [86]:
Pool = Pool(cpu_count())
for _ in tqdm.tqdm(pool.imap_unordered(compute_neighbors, range(len(labels))), total=len(labels)):
    pass
pool.close()
pool.join()




  0%|          | 0/106770 [00:00<?, ?it/s]


  0%|          | 1/106770 [00:00<13:03:52,  2.27it/s]


  0%|          | 2/106770 [00:00<10:31:27,  2.82it/s]


  0%|          | 7/106770 [00:00<7:41:28,  3.86it/s] 


  0%|          | 9/106770 [00:01<6:54:07,  4.30it/s]


  0%|          | 11/106770 [00:01<5:39:58,  5.23it/s]


  0%|          | 16/106770 [00:01<4:25:42,  6.70it/s]


  0%|          | 18/106770 [00:01<3:54:28,  7.59it/s]


  0%|          | 20/106770 [00:01<3:47:41,  7.81it/s]


  0%|          | 24/106770 [00:02<3:07:53,  9.47it/s]


  0%|          | 26/106770 [00:02<3:04:29,  9.64it/s]


  0%|          | 28/106770 [00:02<3:05:50,  9.57it/s]


  0%|          | 30/106770 [00:02<2:40:10, 11.11it/s]


  0%|          | 32/106770 [00:02<2:42:22, 10.96it/s]


  0%|          | 34/106770 [00:03<2:39:57, 11.12it/s]


  0%|          | 36/106770 [00:03<2:56:11, 10.10it/s]


  0%|          | 39/106770 [00:03<2:22:00, 12.53it/s]


  0%|          | 41/106770 [00:03<2:35:28, 11.44it/s]


 

KeyboardInterrupt: 

Process ForkPoolWorker-60:
Traceback (most recent call last):
  File "/usr/lib64/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Process ForkPoolWorker-58:
Process ForkPoolWorker-61:
Process ForkPoolWorker-56:
Process ForkPoolWorker-57:
Process ForkPoolWorker-59:
Process ForkPoolWorker-69:
Process ForkPoolWorker-65:
  File "/usr/lib64/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Process ForkPoolWorker-68:
Process ForkPoolWorker-62:
Process ForkPoolWorker-67:
Process ForkPoolWorker-55:
Process ForkPoolWorker-64:
Process ForkPoolWorker-66:
Traceback (most recent call last):
Process ForkPoolWorker-63:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib64/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Process ForkPoolWorker-70:
Traceback

  File "/usr/lib64/python3.7/multiprocessing/managers.py", line 796, in _callmethod
    kind, result = conn.recv()
  File "/usr/lib64/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib64/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "<string>", line 2, in put
  File "/usr/lib64/python3.7/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "<string>", line 2, in put
  File "_cython_magic_5655e65519d21967c61495ef0051cf1a.pyx", line 50, in _cython_magic_5655e65519d21967c61495ef0051cf1a.compute_neighbors
  File "/usr/lib64/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "<string>", line 2, in put
KeyboardInterrupt
  File "<string>", line 2, in put
  File "/usr/lib64/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  Fi

In [ ]:
bar = progressbar.ProgressBar()
for _ in bar(range(Q.qsize())):
    i, j = Q.get()
    A[i].append(j)
    A[j].append(i)
    G.add_edge(i, j)

In [ ]:
print(sum([len(row) for row in A])/N)

In [ ]:
degree_sequence = sorted(map(lambda t: t[1], G.degree()), reverse=False)
degreeCount = collections.Counter(degree_sequence)
deg, cnt = zip(*degreeCount.items())

In [ ]:
IG = nx.Graph()
with open('data') as f:
    Idata = f.readlines()
for line in Idata[8:]:
    a, b, _ = line.split()
    IG.add_node(a)
    IG.add_node(b)
    IG.add_edge(a, b)
Idegree_sequence = sorted(map(lambda t: t[1], IG.degree()), reverse=False)
IdegreeCount = collections.Counter(Idegree_sequence)
Ideg, Icnt = zip(*IdegreeCount.items())
#Icnt = [int(val*N/len(Idata)) for val in Icnt]

In [ ]:
fig, ax = plt.subplots()
#plt.loglog(deg, cnt, color='b')
plt.loglog(deg, cnt, 'b', Ideg, Icnt, 'r')

plt.title("Degree Histogram")
plt.ylabel("Count")
plt.xlabel("Degree")
#ax.set_xticks([d + 0.4 for d in deg])
#ax.set_xticklabels(deg)
plt.show()

In [ ]:
cnt[10], deg[10], Icnt[9], Ideg[9]

In [ ]:
print(nx.average_clustering(G), nx.average_clustering(IG))

ValueError: Buffer dtype mismatch, expected 'long long' but got 'int'